# Set the scene

In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.svm import LinearSVC
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report, confusion_matrix, auc, average_precision_score, balanced_accuracy_score
import DataPreparation.Prepare_Ultra_Data as DP
import Classification.shallow_classifiers as shallow

# Fetching the UltraSuit Dataset

In [ ]:
!mkdir -p ../Datasets/UltraSuite/

In [ ]:
!rsync -av ultrasuite-rsync.inf.ed.ac.uk::ultrasuite/labels-uxtd-uxssd-upx .      

In [ ]:
!rsync -av --include="*/" --include="*.wav" --exclude="*" ultrasuite-rsync.inf.ed.ac.uk::ultrasuite/core-uxtd ../Datasets/UltraSuite/
!rsync -av --include="*/" --include="*.wav" --exclude="*" ultrasuite-rsync.inf.ed.ac.uk::ultrasuite/core-uxssd ../Datasets/UltraSuite/
!rsync -av --include="*/" --include="*.wav" --exclude="*" ultrasuite-rsync.inf.ed.ac.uk::ultrasuite/core-upx ../Datasets/UltraSuite/        

# Extract features

UXTD Dataset (Typically Development 58 Childern)

In [ ]:
#Get the start and end time of each CHILD segment from the speaker diarization TextGrid files of each dataset
#You can specify which session and task that will be included
uxtd_dWaves_Segments = DP.Select_Data('../Datasets/UltraSuite/core-uxtd', '../Datasets/UltraSuite/labels-uxtd-uxssd-upx/uxtd/speaker_labels/TG/', Sessions='', Tasks=['A','B','C'])
#dump the segments to text file
DP.Write_Wave_Segments_To_File(uxtd_dWaves_Segments,'uxtd_segments.csv')
#Run SMILExtract on each segment to extract the desired features
#Features defined by the config file passed to the function
## GeMAPs (62 features per segment)
DP.Extract_Features_openSmile('uxtd_segments.csv',sConfig_File='openSmile/config/gemaps/GeMAPSv01a.conf',sSegment_Level_csv_File='uxtd_output_GeMAPs.csv')
## eGeMAPs (88 features per segment)
DP.Extract_Features_openSmile('uxtd_segments.csv',sConfig_File='openSmile/config/gemaps/eGeMAPSv01a.conf',sSegment_Level_csv_File='uxtd_output_eGeMAPs.csv')

UXSSD Dataset (8 SSD children)

In [ ]:
#Get the start and end time of each CHILD segment from the speaker diarization TextGrid files of each dataset
#You can specify which session and task that will be included
uxssd_dWaves_Segments = DP.Select_Data('../Datasets/UltraSuite/core-uxssd', '../Datasets/UltraSuite/labels-uxtd-uxssd-upx/uxssd/speaker_labels/TG/', Sessions='BL1', Tasks=['A','B','C'])
#dump the segments to text file
DP.Write_Wave_Segments_To_File(uxssd_dWaves_Segments,'uxssd_segments.csv')
#Run SMILExtract on each segment to extract the desired features
#Features defined by the config file passed to the function
## GeMAPs (62 features per segment)
DP.Extract_Features_openSmile('uxssd_segments.csv',sConfig_File='openSmile/config/gemaps/GeMAPSv01a.conf',sSegment_Level_csv_File='uxssd_output_GeMAPs.csv')
## eGeMAPs (88 features per segment)
DP.Extract_Features_openSmile('uxssd_segments.csv',sConfig_File='openSmile/config/gemaps/eGeMAPSv01a.conf',sSegment_Level_csv_File='uxssd_output_eGeMAPs.csv')

UPX Dataset (20 SSD children)

In [ ]:
#Get the start and end time of each CHILD segment from the speaker diarization TextGrid files of each dataset
#You can specify which session and task that will be included
upx_dWaves_Segments = DP.Select_Data('../Datasets/UltraSuite/core-upx', '../Datasets/UltraSuite/labels-uxtd-uxssd-upx/upx/speaker_labels/TG/', Sessions='BL1', Tasks=['A','B','C'])
#dump the segments to text file
DP.Write_Wave_Segments_To_File(upx_dWaves_Segments,'upx_segments.csv')
#Run SMILExtract on each segment to extract the desired features
#Features defined by the config file passed to the function
## GeMAPs (62 features per segment)
DP.Extract_Features_openSmile('upx_segments.csv',sConfig_File='openSmile/config/gemaps/GeMAPSv01a.conf',sSegment_Level_csv_File='upx_output_GeMAPs.csv')
## eGeMAPs (88 features per segment)
DP.Extract_Features_openSmile('upx_segments.csv',sConfig_File='openSmile/config/gemaps/eGeMAPSv01a.conf',sSegment_Level_csv_File='upx_output_eGeMAPs.csv')

# Building SVM Model

Prepare training/validating

In [ ]:
#Import data to pandas DataFram
#GeMAPs
uxtd_data = pd.read_csv('uxtd_output_GeMAPs.csv',sep=';')
uxssd_data = pd.read_csv('uxssd_output_GeMAPs.csv',sep=';')
upx_data = pd.read_csv('upx_output_GeMAPs.csv',sep=';')

In [ ]:
#eGeMAPs
#uxtd_data = pd.read_csv('uxtd_output_eGeMAPs.csv',sep=';')
#uxssd_data = pd.read_csv('uxssd_output_eGeMAPs.csv',sep=';')
#upx_data = pd.read_csv('upx_output_eGeMAPs.csv',sep=';')

In [ ]:
#Create CV partitioning based on the split_file.csv, the function also convert the pandas DataFram to numpy arrays
#Return X and y and the CV partitions in addition to a string array with the speaker of each sample
X, y, Spkrs, CV = DP.Split_Wavs_Train_Test_From_Speaker_List([('UXTD',uxtd_data),('UXSSD',uxssd_data),('UPX',upx_data)], 'split_file.csv',nDim=62) #nDim=88 for eGeMAPs

In [ ]:
#Apply Feature selection
estimator = SVC(kernel='linear',C=10)
selector = shallow.Feature_Selection(estimator,X,y,cv=CV)
X = selector.transform(X)

In [ ]:
#This function run GridSearch on multiple shallow classifiers (curently only SVM), and search over range of values for different parameters
#This function also perform normalization (currently MaxMin normalization with range [0,1])
#The function fit the normalization on the training part of the current CV partion and apply on the validation part
#The function use UAR (unweighted Average Recall) as its metrics (balanced_accuracy_score)
aTrainedModels = shallow.GridSearchShallow(X,y,CV,bSave_Model=True,prefix='_CV_GeMAPs')

In [ ]:
#Calculate different metrics on both segment and speaker levels of a specific estimator and return reference and predict labels on both segment and speaker level (if SpkrLevel != 'None') otherwise only the segment level label will be returned.
#The desired metrics could be passed to the function via the scorers argument.
#SpkrLevel can take 3 values: - None: where no speaker level score will be calculated
#                             - average: the features of each speaker will be averaged and then predicted as one sample.
#                             - major: each segment will be predicted and then if the majority of segments predicted as TD the speaker will be predicted as TD or SSD otherwise.
y_ref, y_predict_trans, y_ref_spk, y_predict_spk = shallow.Score_CV(aTrainedModels[0].best_estimator_,X,y,cv=CV,aSpeaker_List=np.asarray(Spkrs,dtype=str), SpkrLevel='major', scorers=[('balanced_accuracy_score',balanced_accuracy_score),('accuracy_score', accuracy_score),('confusion_matrix',confusion_matrix)])